# "Vanilla" U-Net Quickstart

*Yuxi Long*

## 1. Introduction

U-Net is a neural network structure that has shown great results in biomedical image segmentation. See [this paper](https://www.nature.com/articles/s41592-018-0261-2). This repository contains an implementation of this network. An alternate version of U-Net, the Siamese U-Net, is also included in this package and its documentation can be found at [using_siam_unet.ipynb](https://github.com/danihae/bio-image-unet/blob/master/biu/using_siam_unet.ipynb).

If you need help using a function, you can always try running `help(whichever_interesting_function)` or just look at the source code. If you need help using a class (one that is directly under the `biu.unet` directory), trying to understand the examples in this notebook probably will be more helpful than finding the documentation of that function.

IMPORTANT: Two packages that depend on your hardware need to be installed manually before running biu. To install CUDA 11.1 which is officially supported by PyTorch, navigate to [its installation page](https://developer.nvidia.com/cuda-11.1.1-download-archive) and follow the instructions onscreen. Because PyTorch depends on your CUDA installation version, it will need to be installed manually as well, through [the official PyTorch website](https://pytorch.org/get-started/locally/). Select the correct distribution of CUDA on this webpage and run the command in your terminal. biu doesn't depend on a specific version of CUDA and has been tested with PyTorch 1.7.0+.

Finally, to import the U-Net package, write `import biu.unet as unet`.

### Basics of U-Net

U-Net takes in one frame as input and tries to predict the labels as its output. Theoretically, the labels can be whatever you want -- it can be a certain type of tissue, or it can be cell boundaries.

To train the U-Net, we need to first do the work of drawing the cell boundaries (or any other kind of label you want) ourselves, so the machine can learn how to draw the cell boundaries given the frames. Example training data can be found at <https://filedn.eu/lKfS794F9UgX7PDuBQcfChB/DeepTissue/>. 

You can use any drawing software to create the training data. We have found [Autodesk Sketchbook](https://www.autodesk.com/products/sketchbook/overview) or [GIMP](https://www.gimp.org/) to be quite useful. Just make sure to use a tablet with stylus and turn on a podcast while you draw. To create the training data, we first isolate the frame we want either using `biu.helpers.extract_frame_of_movie`, or use gimp or whatever magical tool you have. Open the image in your drawing software, decrease its opacity, add another layer on top of the image, and start drawing your labels (what you want the network to predict). You are encouraged to use the pen tool with color black and radius 1 in your drawing software.

After you have finished drawing labels of the entire image, remove (or make invisible) the image layer, and you are just left with the label layer. Export your label, convert it to grayscale using any tool you can find (we have found <https://online-photo-converter.com/black-and-white-image> to be handy), name it the same as the image, and move on to the next frame you wish to create a label for. You don't need to create a label for every frame, but theoretically the more frames you have, the better your neural network will learn. Always feel free to train the network and see what output you get even if you don't think you have enough labels.

Once you are done creating labels, move on to the next step.

## 2. Data preparation

For the "vanilla" U-Net, this step is simple.

When you are done converting your labels to grayscale, organize the labels and images in a way similar to this shown. An example can be found in `training_data.zip`. 

```
training_data/lateral_epidermis
├── yokogawa
│   ├── image
│   │   ├── 1.tif
│   │   ├── 2.tif
│   │   ├── 3.tif
│   │   ├── 5.tif
│   │   ├── 7.tif
│   │   └── 83.tif
│   └── label
│       ├── 1.tif
│       ├── 2.tif
│       ├── 3.tif
│       ├── 5.tif
│       ├── 7.tif
│       └── 83.tif

```

## 3. Training

Training is simple. For example:

In [ ]:
from biu.unet import *

dataset = 'lateral_epidermis/yokogawa' # example using sample dataset
base_dir = './' 

# path to training data (images and labels with identical names in separate folders)
dir_images = f'{base_dir}/training_data/{dataset}/image/'
dir_masks = f'{base_dir}/training_data/{dataset}/label/'

print('starting to create training dataset')
# create training data set
data = DataProcess([dir_images, dir_masks], data_path='./data', dilate_mask=0, aug_factor=10, create=False, invert=False, clip_thres=(0.2, 99.8), dim_out=(256, 256), shiftscalerotate=(0, 0, 0))

save_dir = f'{base_dir}/models/yokogawa_lateral_epidermis'
# create trainer
training = Trainer(data ,num_epochs=500 ,batch_size=12, load_weights=False, lr=0.0001, n_filter=32, save_iter=True, save_dir=save_dir)

training.start()

Note here that the value of the `n_filter` parameter is set to `32`. The network won't break with a different value of this, but you need to use the same value for the Predict part.

## 4. Predict

Predicting is simple as well. Just swap in the parameters

In [ ]:
# load package
from biu.unet import *
import os
os.nice(10)
from  biu.siam_unet.helpers import tif_to_mp4

base_dir = './'
out_dir = f'{base_dir}/predicted_out'
model = f'{base_dir}/models/yokogawa_lateral_epidermis/model_epoch_100.pth'

tif_file = f'{base_dir}/training_data/test_data/new_microscope/21C04_shgGFP_kin_2_Pos4.tif'

result_file = f'{out_dir}/siam_bce_amnio_100_epochs_21C04_shgGFP_kin_2_Pos4.tif'
out_mp4_file = result_file[:-4] + '.mp4'

print('starting to predict file')
# predict file 
predict = Predict(tif_file, result_file, model, invert=False, resize_dim=(512, 512), n_filter=32)
# convert to mp4
tif_to_mp4.convert_to_mp4(result_file, output_file=out_mp4_file, normalize_to_0_255=True)

# Appendix: An annotated structure of the siam_unet package

Below is an annotated structure of the siam_unet package. Use `help(function)` to read the docstring of each function for a better understanding.

```
Package                                         Use

.
├── __init__.py
├── data.py                                     dataloader script
├── siam_unet.py                                Siam U-Net model
├── train.py                                    training script
├── losses.py                                   loss functions
├── predict.py                                  prediction script
├── helpers                                     helper functions (usually not 
                                                        so useful except the 
                                                        ones mentioned in this notebook)
                                                        
│   ├── average_tifs.py                             averages a list of tiff files
│   ├── create_pixel_value_histogram.py             creates histograms for the 
                                                        pixel values in tif 
                                                        files. Useful for 
                                                        debugging during training
│   ├── cuda_test.py                                tests cuda functionality
│   ├── extract_frame_of_movie.py                   extract a certain frame of a 
                                                        tif movie 
│   ├── find_frame_of_image.py                      finds the frame number of 
                                                        a given query image 
                                                        within search_space.
│   ├── generate_plain_image.py                     generates a plain image
│   ├── generate_siam_unet_input_imgs.py            generates a coupled image 
                                                        for Siam U-Net training
│   ├── low_mem_tif_utils.py                        utilities for handling tif 
                                                        files with low memory 
                                                        usage
│   ├── threshold_images.py                         thresholds each frame of a 
                                                        tif movie
│   ├── tif_to_mp4.py                               uses ffmpeg to convert a tif 
                                                        movie to mp4
│   └── util.py                                     various utilities. see docstring

```